# 2. Run Nucleus Segmentation Model 

gg-napari-env

In [1]:
from napari_czifile2 import napari_get_reader
from cellpose import models, core
from cellpose.io import logger_setup
import torch 
import numpy as np
import numpy as np
import os 
from plotting import * 
import warnings
warnings.filterwarnings("ignore")
use_GPU = core.use_gpu()
torch.cuda.empty_cache()
print('>>> GPU activated? %d'%use_GPU)
logger_setup()

Initializing plotting package
>>> GPU activated? 1
2025-05-24 11:14:28,751 [INFO] WRITING LOG OUTPUT TO /home/tracking/.cellpose/run.log
2025-05-24 11:14:28,752 [INFO] 
cellpose version: 	3.1.0 
platform:       	linux 
python version: 	3.12.8 
torch version:  	2.5.1+cu124


(<Logger cellpose.io (INFO)>, PosixPath('/home/tracking/.cellpose/run.log'))

In [2]:
raw_data_dirs = os.listdir('../../../RNA-FISH-raw-data/')
raw_data_dirs

['20250508 P14 T 9E89-b1-546 9E99-B2-647 DAPI',
 '20250325 6 9e198-b3-488 9e197-b1-546 9e196-b2-647 dapi',
 '20250307 B1 P14 U34-B3-546 Chymotrypsin-B2-647 DAPI',
 '20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI',
 '20250328 5 P14 LOC603-b3-488 9E108-b1-546 9E116-b2-647 DAPI',
 '20250325 5 p14 g1-b1-546 lnc7-b2-647 dapi',
 '20250508 P14 T 9E99-b2-594 LOC105286072-B5-647 DAPI',
 '20250307 C2 stacks',
 '20250324 A1 stacks',
 '20250328 2 P14 R2-b3-488 Q1-b1-546 Lnc6-b2-647 DAPI',
 '20250523 C P14 T U54-647 Lnc4-546 DAPI',
 '20250523 A P14 T 9E213-546 9E214-488 Intergenic-647 DAPI',
 '20250325 4 p14 u34-b3-488 lnc4-b1-546 u21-b5-647 dapi',
 '20250328 3 P14 Lnc3-b3-488 L16-b2-594 Lnc2-b5-647 DAPI',
 '20250324 A2 stacks',
 '20250306 C1 stacks',
 '20250328 4 P14 9E129-b3-488 LOC104-b1-546 9E116-b2-647 dapi',
 '20250523 B P14 T 9E88-546 Lnc-647 DAPI']

In [3]:
input = '20250523 C'
input = [d for d in raw_data_dirs if input in d][0]
print(f'Using {input} as input directory')
input_dir = f'../../../RNA-FISH-raw-data/{input}/'
assert os.path.exists(input_dir), 'Input directory does not exist'
czi_files = [f for f in os.listdir(input_dir) if f.endswith('.czi')]
print(f"Found {len(czi_files)} czi files in {input_dir}")
print(czi_files)

Using 20250523 C P14 T U54-647 Lnc4-546 DAPI as input directory
Found 4 czi files in ../../../RNA-FISH-raw-data/20250523 C P14 T U54-647 Lnc4-546 DAPI/
['20250523 C sample 4 stack.czi', '20250523 C sample 2 stack.czi', '20250523 C sample 3 stack.czi', '20250523 C sample 1 stack.czi']


In [4]:
# Load trained model 
model_path = '../cellpose-model-training/model_B_only_OSNs/models/model_B_only_OSNs'
model = models.CellposeModel(gpu=True, pretrained_model=model_path)

2025-05-24 11:14:45,715 [INFO] ** TORCH CUDA version installed and working. **
2025-05-24 11:14:45,715 [INFO] >>>> using GPU (CUDA)
2025-05-24 11:14:45,759 [INFO] >>>> loading model ../cellpose-model-training/model_B_only_OSNs/models/model_B_only_OSNs
2025-05-24 11:14:46,075 [INFO] >>>> model diam_mean =  17.000 (ROIs rescaled to this size during training)
2025-05-24 11:14:46,077 [INFO] >>>> model diam_labels =  29.437 (mean diameter of training ROIs)


In [5]:
results_dir = f'../results/{input}'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
for f in czi_files[:]:
    print(f"Processing {f}")
    file_path = os.path.join(input_dir, f)
    reader = napari_get_reader(file_path)
    if reader is not None:
        layer_data = reader(file_path)
        image_data, metadata, layer_type = layer_data[0]
        # Remove singleton dimension 
        image_data = np.squeeze(image_data)  
        print("Metadata:", metadata)
        print("Image shape:", image_data.shape)  
        print(f"Number of NaN pixels: {np.sum(np.isnan(image_data))}")

        pixel_x = metadata['scale'][2]
        print(f"Pixel size: {pixel_x} um")

        xy_resolution = image_data.shape[-1] 
        print(f"XY resolution: {xy_resolution} pixels")

        nucleus_diameter = 3 / pixel_x
        print(f"Nucleus diameter: {nucleus_diameter} pixels") 
        
        z_slices = image_data.shape[0]
        print(f"Z slices: {z_slices}")

        z_um = metadata['scale'][1]
        print(f"Z resolution: {z_um} um")

        dapi_index = metadata['name'].index([name for name in metadata['name'] if 'DAPI' in name][0])
        print(f"DAPI channel index: {dapi_index}")

        non_dapi_channels = [(i, metadata['name'][i]) for i in range(len(metadata['name'])) if i != dapi_index]
        print(f"Non-DAPI channels: {non_dapi_channels}")

        all_rois = np.zeros_like(image_data[:, dapi_index, :, :], dtype=np.uint16)
        for z in range(z_slices):
            image_data_z = image_data[z, dapi_index, :, :]
            print(f"Z={z} | shape={image_data_z.shape} | dtype={image_data_z.dtype} | min={image_data_z.min()} | max={image_data_z.max()}")
            try:
                rois, flows, styles = model.eval(image_data_z, diameter=nucleus_diameter)
                print(f"Identified {rois.max()} nuclei at z={z}")
            except Exception as e:
                print(f"Error at z={z}: {e}")
                continue
            print(f"Identified {rois.max()} nuclei")
            all_rois[z, :, :] = rois

        # Save the masks to a file 
        all_rois_path = os.path.join(results_dir, f.replace('.czi', f'_rois.npy'))
        np.save(all_rois_path, all_rois)
        print(f"ROIs saved to {all_rois_path}")

Processing 20250523 C sample 4 stack.czi
Metadata: {'rgb': False, 'channel_axis': 2, 'translate': (0.0, 0.0, 0.0, 0.0), 'scale': (1.0, 1.0, 0.0974884033203125, 0.0974884033203125), 'contrast_limits': None, 'name': ['AF546-T1', 'DAPI-T2', 'AF647-T2']}
Image shape: (94, 3, 2048, 2048)
Number of NaN pixels: 0
Pixel size: 0.0974884033203125 um
XY resolution: 2048 pixels
Nucleus diameter: 30.77289090624511 pixels
Z slices: 94
Z resolution: 1.0 um
DAPI channel index: 1
Non-DAPI channels: [(0, 'AF546-T1'), (2, 'AF647-T2')]
Z=0 | shape=(2048, 2048) | dtype=uint8 | min=0 | max=255
Identified 6 nuclei at z=0
Identified 6 nuclei
Z=1 | shape=(2048, 2048) | dtype=uint8 | min=0 | max=255
Identified 28 nuclei at z=1
Identified 28 nuclei
Z=2 | shape=(2048, 2048) | dtype=uint8 | min=0 | max=255
Identified 55 nuclei at z=2
Identified 55 nuclei
Z=3 | shape=(2048, 2048) | dtype=uint8 | min=0 | max=255
Identified 92 nuclei at z=3
Identified 92 nuclei
Z=4 | shape=(2048, 2048) | dtype=uint8 | min=0 | max=255

## Run on all files

In [ ]:
raw_data_dirs = os.listdir('../../../RNA-FISH-raw-data/')
raw_data_p14 = [d for d in raw_data_dirs if '14' in d]
raw_data_p14

['20250307 B1 P14 U34-B3-546 Chymotrypsin-B2-647 DAPI',
 '20250328 1 P14 T79-intergenic-b2-647 T79-exonic-b1-546 DAPI',
 '20250328 5 P14 LOC603-b3-488 9E108-b1-546 9E116-b2-647 DAPI',
 '20250325 5 p14 g1-b1-546 lnc7-b2-647 dapi',
 '20250328 2 P14 R2-b3-488 Q1-b1-546 Lnc6-b2-647 DAPI',
 '20250328 4 P14 9E129-b3-488 LOC104-b1-546 9E116-b2-647 dapi',
 '20250325 4 p14 u34-b3-488 lnc4-b1-546 u21-b5-647 dapi',
 '20250328 3 P14 Lnc3-b3-488 L16-b2-594 Lnc2-b5-647 DAPI']

In [8]:
for input in raw_data_p14: 

    input_dir = f'../raw-data/{input}/'
    assert os.path.exists(input_dir), 'Input directory does not exist'
    czi_files = [f for f in os.listdir(input_dir) if f.endswith('.czi')]
    print(f"Found {len(czi_files)} czi files in {input_dir}")
    print(czi_files)

    results_dir = f'../results/{input}'
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
        
    for f in czi_files[:]:
        print(f"Processing {f}")
        file_path = os.path.join(input_dir, f)
        reader = napari_get_reader(file_path)
        if reader is not None:
            layer_data = reader(file_path)
            image_data, metadata, layer_type = layer_data[0]
            # Remove singleton dimension 
            image_data = np.squeeze(image_data)  
            print("Metadata:", metadata)
            print("Image shape:", image_data.shape)  
            print(f"Number of NaN pixels: {np.sum(np.isnan(image_data))}")

            pixel_x = metadata['scale'][2]
            print(f"Pixel size: {pixel_x} um")

            xy_resolution = image_data.shape[-1] 
            print(f"XY resolution: {xy_resolution} pixels")

            nucleus_diameter = 3 / pixel_x
            print(f"Nucleus diameter: {nucleus_diameter} pixels") 
            
            z_slices = image_data.shape[0]
            print(f"Z slices: {z_slices}")

            z_um = metadata['scale'][1]
            print(f"Z resolution: {z_um} um")

            dapi_index = metadata['name'].index([name for name in metadata['name'] if 'DAPI' in name][0])
            print(f"DAPI channel index: {dapi_index}")

            non_dapi_channels = [(i, metadata['name'][i]) for i in range(len(metadata['name'])) if i != dapi_index]
            print(f"Non-DAPI channels: {non_dapi_channels}")

            all_rois = np.zeros_like(image_data[:, dapi_index, :, :], dtype=np.uint16)
            for z in range(z_slices):
                image_data_z = image_data[z, dapi_index, :, :]
                print(f"Z={z} | shape={image_data_z.shape} | dtype={image_data_z.dtype} | min={image_data_z.min()} | max={image_data_z.max()}")
                try:
                    rois, flows, styles = model.eval(image_data_z, diameter=nucleus_diameter)
                    print(f"Identified {rois.max()} nuclei at z={z}")
                except Exception as e:
                    print(f"Error at z={z}: {e}")
                    continue
                print(f"Identified {rois.max()} nuclei")
                all_rois[z, :, :] = rois

            # Save the masks to a file 
            all_rois_path = os.path.join(results_dir, f.replace('.czi', f'_rois.npy'))
            np.save(all_rois_path, all_rois)
            print(f"ROIs saved to {all_rois_path}")

Found 5 czi files in ../raw-data/20250307 B1 P14 U34-B3-546 Chymotrypsin-B2-647 DAPI/
['20250307 B1 Sample 5 Stack.czi', '20250307 B1 Sample 4 Stack.czi', '20250307 B1 Sample 1 Stack.czi', '20250307 B1 Sample 3 Stack.czi', '20250307 B1 Sample 2 Stack.czi']
Processing 20250307 B1 Sample 5 Stack.czi
Metadata: {'rgb': False, 'channel_axis': 2, 'translate': (0.0, 0.0, 0.0, 0.0), 'scale': (1.0, 0.5, 0.0974884033203125, 0.0974884033203125), 'contrast_limits': None, 'name': ['AF546-T1', 'DAPI-T2', 'AF647-T2']}
Image shape: (202, 3, 2048, 2048)
Number of NaN pixels: 0
Pixel size: 0.0974884033203125 um
XY resolution: 2048 pixels
Nucleus diameter: 30.77289090624511 pixels
Z slices: 202
Z resolution: 0.5 um
DAPI channel index: 1
Non-DAPI channels: [(0, 'AF546-T1'), (2, 'AF647-T2')]
Z=0 | shape=(2048, 2048) | dtype=uint8 | min=0 | max=255
2025-04-07 08:42:27,658 [INFO] No cell pixels found.
Identified 0 nuclei at z=0
Identified 0 nuclei
Z=1 | shape=(2048, 2048) | dtype=uint8 | min=0 | max=255
2025